# Análisis Encuesta Origen-Destino -  Limpieza de Datos & Cálculo de Indicadores

## Chile - Santiago - 2012

#### Elaborado por Paula Vásquez-Henríquez, Ariel López, Genaro Cuadros, Exequiel Gaete, Alba Vásquez y Juan Correa

## Google colab

Para ejecutar este notebook en Colab, primero descomenten y ejecuten las siguientes 3 celdas. Luego de ejecutar la notebook se reiniciará.

In [ ]:
'''
!pip3 uninstall matplotlib -y
!pip install -q condacolab
import condacolab
condacolab.install()
'''

In [ ]:
'''
!git clone https://github.com/zorzalerrante/aves.git aves_git
!mamba env update --name base --file aves_git/environment-colab.yml
'''

In [ ]:
'''
# Montando datos desde Google Drive
from google.colab import drive
drive.mount('/content/drive')
'''

## Instalando e importando librerías

In [ ]:
# Estas librerías se deben instalar sólo si se está ejecutando localmente
!pip3 install matplotlib
!pip3 install seaborn
!pip3 install sklearn

In [ ]:
#Estas librerías se deben instalar tanto corriendolo localmente como en Google Colab
!pip3 install geopandas
!pip3 install haversine
!pip3 install pandasql
!pip3 install openpyxl

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import geopandas as gpd
import warnings
import haversine as hs
import shapely
from sklearn.preprocessing import normalize
import datetime
from pyproj import CRS
from shapely.geometry import Point

In [ ]:
# Si se está en google colab, reemplazar por path de Drive
data_path = 'C:/Users/Usuario/Documents/GitHub/enmodo/'

In [ ]:
import sys

# Si se está en google colab, reemplazar por path donde tiene la carpeta "scripts"
sys.path.insert(1, data_path +'scripts')

import eod_analysis as eod

In [ ]:
def convert_datatype(df, lista_columnas):
    for column in lista_columnas:
        df[column] = df[column].str.replace(",", ".").astype(float)
    return df

In [ ]:
def decode_column(df, fname, col_name, index_col='id', value_col=None, sep=';', encoding='utf-8'):
    '''
    param :df: DataFrame del que leeremos una columna.
    param :fname: nombre del archivo que contiene los valores a decodificar.
    param :col_name: nombre de la columna que queremos decodificar.
    param :index_col: nombre de la columna en el archivo @fname que tiene los índices que codifican @col_name
    param :value_col: nombre de la columna en el archivo @fname que tiene los valores decodificados
    param :sep: carácter que separa los valores en @fname. 
    param :encoding: identificación del _character set_ que utiliza el archivo. Usualmente es utf-8, si no funciona,
                     se puede probar con iso-8859-1.
    '''
    if value_col is None:
        value_col = 'value'
        
    values_df = pd.read_csv(fname, sep=sep, index_col=index_col, names=[index_col, value_col], header=0,
                            dtype={index_col: np.float64}, encoding=encoding)
    
    src_df = df.loc[:,(col_name,)]
    
    return src_df.join(values_df, on=col_name)[value_col]

In [ ]:
def imputar_coordenadas_centroide_zat(df, latitud, longitud, zat, zona_shp):
    columns = df.columns
    df['mask'] = df[latitud].isnull() | df[longitud].isnull()
    working_df = pd.merge(df, city_shp[[zona_shp, 'x_coord', 'y_coord']], left_on=zat, right_on=zona_shp, how='left')
    working_df[latitud] = np.where(working_df['mask']==True, working_df['x_coord'], working_df[latitud])
    working_df[longitud] = np.where(working_df['mask']==True, working_df['y_coord'], working_df[longitud])
    
    return working_df[columns]

In [ ]:
def mapear_binarias(row, column):
    if row[column] == 'N':
        return 'No'
    elif row[column] == 'S':
        return 'Si'
    else:
        return 'Sin dato'

In [ ]:
def mapear_vacios(row, column):
    if pd.isna(row[column])==True:
        return 'Sin información'
    else:
        return row[column]

### Caracterización de los datos

Los datos utilizados en este cuaderno corresponden a los resultados de la Encuesta Origen-Destino de Santiago, Chile del 2012. 
A partir de estos datos se calcularan indicadores en tres niveles: de Cantidad de Viajes, de Tiempo de Viajes, y de Distancia de Viajes.

### Importando datos

En esta sección, importamos todos los datos necesarios para el cálculo de indicadores. 
Para el caso de Santiago 2012, los archivos son de csv.

Cabe destacar que para esta EOD tenemos data a nivel de:
- Viajes
- Etapas
- Personas
- Hogares

Sin embargo, los indicadores en este caso se calculan a nivel de **VIAJE Y PERSONA**.

In [ ]:
# Matriz de viajes
data_viajes = pd.read_csv(data_path + 'santiago/source-csv/viajes.csv', sep=";")
data_viajes = convert_datatype(data_viajes, ['OrigenCoordX', 'OrigenCoordY', 'DestinoCoordX', 'DestinoCoordY', 'FactorLaboralNormal', 'FactorSabadoNormal', 'FactorDomingoNormal', 'FactorLaboralEstival', 'FactorFindesemanaEstival'])

In [ ]:
distancia_viajes = pd.read_csv(data_path + 'santiago/source-csv/DistanciaViaje.csv', sep=";")

In [ ]:
data_viajes = pd.merge(data_viajes, distancia_viajes, on='Viaje', how='left')

In [ ]:
# Matriz de personas
data_personas = pd.read_csv(data_path + 'santiago/source-csv/personas.csv', sep=";", encoding='latin-1')
data_personas = convert_datatype(data_personas, ['Factor_LaboralNormal', 'Factor_SabadoNormal', 'Factor_DomingoNormal', 'Factor_LaboralEstival', 'Factor_FindesemanaEstival', 'Factor'])

In [ ]:
# Matriz de hogares
data_hogares = pd.read_csv(data_path + 'santiago/source-csv/Hogares.csv', sep=";", encoding='latin-1')

In [ ]:
# Shapefile de la ciudad
city_shp = gpd.read_file(data_path + 'santiago/source-shp/Zonificacion_EOD2012.shp', crs='EPSG:32719')

In [ ]:
from os import listdir
from os.path import isfile, join
filenames = [f for f in listdir(data_path + 'santiago/source-shp/') if isfile(join(data_path + 'santiago/source-shp/', f))]
filenames = [y for y in sorted(filenames) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
dbf, prj, shp, shx = [filename for filename in filenames]
city_shp = gpd.read_file(data_path + 'santiago/source-shp/'+shp)
print("Shape of the dataframe: {}".format(city_shp.shape))
print("Projection of dataframe: {}".format(city_shp.crs))
city_shp.head()

In [ ]:
# Calculamos los centroides de cada zona
city_shp['x_coord'] = city_shp['geometry'].centroid.x
city_shp['y_coord'] = city_shp['geometry'].centroid.y

In [ ]:
city_shp.crs

### Preparación de los datos

#### Viajes

En esta etapa nos enfocaremos en preparar los datos con respecto a viajes.
En particular, nos enfocamos en limpiar y estandarizar los datos para las columnas que son relevantes para el cálculo de indicadores.


In [ ]:
selected_columns = ['Hogar', 'Persona', 'Viaje', 'Etapas', 'ComunaOrigen', 'ComunaDestino',
       'SectorOrigen', 'SectorDestino', 'ZonaOrigen', 'ZonaDestino',
       'OrigenCoordX', 'OrigenCoordY', 'DestinoCoordX', 'DestinoCoordY',
       'Proposito', 'PropositoAgregado', 'ModoAgregado', 'ModoPriPub', 'HoraIni', 'HoraFin', 'TiempoViaje', 
       'FactorLaboralNormal', 'FactorSabadoNormal',
       'FactorDomingoNormal', 'FactorLaboralEstival',
       'FactorFindesemanaEstival', 'Periodo', 'DistManhattan']
viajes_df = data_viajes[selected_columns]

In [ ]:
viajes_df.shape

In [ ]:
print('Contando valores nulos por atributo')
for column in viajes_df.columns:
    print('{}: {}'.format(column, viajes_df[column].isna().sum()))

Tenemos muchas encuestas sin indicación de origen y destino, cuyo codigo zonal es 0. Los dejaremos fuera de este análisis.

In [ ]:
viajes_df = viajes_df[(viajes_df.ZonaOrigen > 0) & (viajes_df.ZonaDestino > 0)]

In [ ]:
# Llenado de coordenadas para datos vacíos, se imputará centroide del ZAT correspondiente.

In [ ]:
viajes_df = imputar_coordenadas_centroide_zat(viajes_df, 'OrigenCoordX', 'OrigenCoordY', 'ZonaOrigen', 'ID')
viajes_df = imputar_coordenadas_centroide_zat(viajes_df, 'DestinoCoordX', 'DestinoCoordY','ZonaDestino', 'ID')

In [ ]:
# Creando variables de dia habil, no habil, horario punta

In [ ]:
def dia_habil(row):
    if pd.isna(row['FactorLaboralNormal'])==False or pd.isna(row['FactorLaboralEstival'])==False:
        return 'Si'
    else:
        return 'No'

In [ ]:
def dia_nohabil(row):
    if pd.isna(row['FactorSabadoNormal'])==False or pd.isna(row['FactorDomingoNormal'])==False or pd.isna(row['FactorFindesemanaEstival'])==False:
        return 'Si'
    else:
        return 'No'

In [ ]:
#columns_day = ['DIA_HABIL', 'DIA_NOHABIL', 'PICO_HABIL', 'PICO_NOHABIL']
viajes_df['DIA_HABIL'] = viajes_df.apply(lambda row: dia_habil(row), axis=1)
viajes_df['DIA_NOHABIL'] = viajes_df.apply(lambda row: dia_nohabil(row), axis=1)

In [ ]:
viajes_df['Periodo'] = decode_column(viajes_df, data_path + 'santiago/source-csv/Tablas_parametros/Periodo.csv', 'Periodo', 
                                  sep=';', value_col='Periodos')

In [ ]:
punta = ['Punta Tarde (17:31 - 20:30)', 'Punta Mañana 2 (7:31 - 9:00)','Punta Mañana 1 (6:01 - 7:30)']

In [ ]:
# Creando columnas para identificar periodos pico dia habil y dia no habil

In [ ]:
def punta_habil(row):
    if row['Periodo'] in punta and row['DIA_HABIL'] == 'Si':
        return 'Si'
    else:
        return 'No'

In [ ]:
def punta_nohabil(row):
    if row['Periodo'] in punta and row['DIA_NOHABIL'] == 'Si':
        return 'Si'
    else:
        return 'No'

In [ ]:
viajes_df['PICO_HABIL'] = viajes_df.apply(lambda row: punta_habil(row), axis=1)
viajes_df['PICO_NOHABIL'] = viajes_df.apply(lambda row: punta_nohabil(row), axis=1)

In [ ]:
# Para esta encuesta, podemos definir zona en tres niveles: zona, sector y comuna. Definimos viajes intra 
# e inter para los tres niveles

In [ ]:
viajes_df['Intra_Inter_zona'] = viajes_df.apply(lambda row: 'Intra' if row['ZonaOrigen'] == row['ZonaDestino'] else 'Inter', axis=1)
viajes_df['Intra_Inter_sector'] = viajes_df.apply(lambda row: 'Intra' if row['SectorOrigen'] == row['SectorDestino'] else 'Inter', axis=1)
viajes_df['Intra_Inter_comuna'] = viajes_df.apply(lambda row: 'Intra' if row['ComunaOrigen'] == row['ComunaDestino'] else 'Inter', axis=1)

In [ ]:
viajes_df['Proposito'] = decode_column(viajes_df, data_path + 'santiago/source-csv/Tablas_parametros/Proposito.csv', 'Proposito', 
                                  sep=';')

In [ ]:
viajes_df['PropositoAgregado'] = decode_column(viajes_df, data_path + 'santiago/source-csv/Tablas_parametros/PropositoAgregado.csv', 'PropositoAgregado', 
                                  sep=';')

In [ ]:
viajes_df['ModoPriPub'] = decode_column(viajes_df, data_path + 'santiago/source-csv/Tablas_parametros/ModoPriPub.csv', 'ModoPriPub', 
                                  sep=';')

In [ ]:
viajes_df['ModoAgregado'] = decode_column(viajes_df, data_path + 'santiago/source-csv/Tablas_parametros/ModoAgregado.csv', 'ModoAgregado', 
                                  sep=';')

In [ ]:
print('Contando valores nulos por atributo')
for column in viajes_df.columns:
    print('{}: {}'.format(column, viajes_df[column].isna().sum()))

#### Personas y Hogares

En esta etapa nos enfocaremos en preparar los datos con respecto a personas.
En particular, nos enfocamos en limpiar y estandarizar los datos para las columnas que son relevantes para el cálculo de indicadores.

In [ ]:
selected_columns = ['Hogar', 'Persona', 'Sexo', 'Factor_LaboralNormal', 'Factor_SabadoNormal', 'Factor_DomingoNormal',
       'Factor_LaboralEstival', 'Factor_FindesemanaEstival', 'TramoIngreso']
personas_df = data_personas[selected_columns]

In [ ]:
personas_df['TramoIngreso'] = decode_column(personas_df, data_path + 'santiago/source-csv/Tablas_parametros/TramoIngreso.csv', 'TramoIngreso')

In [ ]:
personas_df['Sexo'] = decode_column(personas_df, data_path + 'santiago/source-csv/Tablas_parametros/Sexo.csv', 'Sexo')

In [ ]:
personas_df[['Factor_LaboralNormal', 'Factor_SabadoNormal', 'Factor_DomingoNormal', 'Factor_LaboralEstival', 'Factor_FindesemanaEstival']] = personas_df[['Factor_LaboralNormal', 'Factor_SabadoNormal', 'Factor_DomingoNormal', 'Factor_LaboralEstival', 'Factor_FindesemanaEstival']].fillna(value=0)

In [ ]:
# Calculamos el factor ponderador para las personas

In [ ]:
def ponderador_personas(row):
    return row['Factor_LaboralNormal'] + row['Factor_SabadoNormal'] + row['Factor_DomingoNormal'] + row['Factor_LaboralEstival'] + row['Factor_FindesemanaEstival']

In [ ]:
personas_df['PONDERADOR_CALIBRADO'] = personas_df.apply(lambda row: ponderador_personas(row), axis=1)

In [ ]:
viajes_personas = pd.merge(viajes_df, personas_df, on=['Persona'], how='left')

In [ ]:
print('Contando valores nulos por atributo')
for column in viajes_personas.columns:
    print('{}: {}'.format(column, viajes_personas[column].isna().sum()))

In [ ]:
#Calculando factor ponderador viajes

In [ ]:
def ponderador_viajes(row):
    if pd.isna(row['FactorLaboralNormal'])==False:
        return row['FactorLaboralNormal']*row['Factor_LaboralNormal']
    
    elif pd.isna(row['FactorSabadoNormal'])==False:
        return row['FactorSabadoNormal']*row['Factor_SabadoNormal']
    
    elif pd.isna(row['FactorDomingoNormal'])==False:
        return row['FactorDomingoNormal']*row['Factor_DomingoNormal']
    
    elif pd.isna(row['FactorLaboralEstival'])==False:
        return row['FactorLaboralEstival']*row['Factor_LaboralEstival']
    elif pd.isna(row['FactorFindesemanaEstival'])==False:
        return row['FactorFindesemanaEstival']*row['Factor_FindesemanaEstival']
    else:
        return None

In [ ]:
viajes_personas['PONDERADOR_CALIBRADO_VIAJES'] = viajes_personas.apply(lambda row: ponderador_viajes(row), axis=1)

In [ ]:
print('Contando valores nulos por atributo')
for column in viajes_personas.columns:
    print('{}: {}'.format(column, viajes_personas[column].isna().sum()))

In [ ]:
viajes_personas = viajes_personas[~viajes_personas.PONDERADOR_CALIBRADO_VIAJES.isna()]
viajes_personas.shape

### Generación csv

In [ ]:
viajes_personas.to_csv(data_path + 'santiago/csv/viajes_personas_santiago_2012.csv', index=False)

### Generación geojson

In [ ]:
origenes_viajes = gpd.GeoDataFrame(
    viajes_personas, geometry=gpd.points_from_xy(viajes_personas.OrigenCoordX, viajes_personas.OrigenCoordY, crs='EPSG:32719'))

destinos_viajes = gpd.GeoDataFrame(
    viajes_personas, geometry=gpd.points_from_xy(viajes_personas.DestinoCoordX, viajes_personas.DestinoCoordY, crs='EPSG:32719'))

In [ ]:
origenes_viajes.to_file(data_path + 'santiago/output-geojson/origenes_viajes.geojson', driver='GeoJSON')
destinos_viajes.to_file(data_path + 'santiago/output-geojson/destinos_viajes.geojson', driver='GeoJSON')

### Generación shp

In [ ]:
origenes_viajes.to_file(data_path + 'santiago/output-shp/origenes_viajes.shp', driver='GeoJSON')
destinos_viajes.to_file(data_path + 'santiago/output-shp/destinos_viajes.shp', driver='GeoJSON')

### Parte I: Indicadores de Cantidad de Viajes

En esta primera parte, responderemos algunas preguntas respecto a indicadores de cantidades de viajes realizados, en días hábiles y no hábiles de viaje. Para esto, buscaremos responder las siguientes preguntas:

1. ¿Cuál es la tasa promedio de viajes diarios en transporte público por clasificador económico?
2. ¿Cuál es la tasa promedio de viajes diarios en transporte privado por clasificador económico?
3. ¿Cuál es la razón entre los viajes en transporte público y privado por clasificador socioeconómico?
4. ¿Cuál es la distribución/partición modal de los viajes por clasificador socioeconómico?

In [ ]:
print('Cantidad de viajes mapeados: totales encuesta , total expandido')
viajes_habiles = viajes_personas[viajes_personas.DIA_HABIL=='Si']
print('Total Viajes Habiles: {}'.format(viajes_habiles.shape[0]), viajes_habiles['PONDERADOR_CALIBRADO_VIAJES'].sum())
viajes_nohabiles = viajes_personas[viajes_personas.DIA_NOHABIL=='Si']
print('Total Viajes No Habiles: {}'.format(viajes_nohabiles.shape[0]), viajes_nohabiles['PONDERADOR_CALIBRADO_VIAJES'].sum())

Separamos los viajes hábiles y no hábiles de acuerdo a si son privados o públicos.

In [ ]:
viajes_publico_habiles = viajes_habiles[viajes_habiles.ModoPriPub=='Publico']
viajes_publico_nohabiles = viajes_nohabiles[viajes_nohabiles.ModoPriPub=='Publico']

viajes_privado_habiles = viajes_habiles[viajes_habiles.ModoPriPub=='Privado']
viajes_privado_nohabiles = viajes_nohabiles[viajes_nohabiles.ModoPriPub=='Privado']

In [ ]:
def weighted_mean(df, value_column, weighs_column):
    weighted_sum = (df[value_column] * df[weighs_column]).sum()
    return weighted_sum / df[weighs_column].sum()

In [ ]:
from pandasql import sqldf
def calculate_n_viajes_per_capita(df, df_str, agg_columns_str, agg_columns_lst, id_person, person_weight, trip_weight=None):
    q = "SELECT DISTINCT {}, {}, {} FROM {}".format(id_person, agg_columns_str, person_weight, df_str)
    persons = sqldf(q, globals())
    n_personas = persons.groupby(agg_columns_lst).sum()[[person_weight]].reset_index()
    n_personas[agg_columns_lst[0]] = n_personas[agg_columns_lst[0]].astype(str)
    n_viajes = df.groupby(agg_columns_lst).sum()[[trip_weight]].reset_index()
    n_viajes[agg_columns_lst[0]] = n_viajes[agg_columns_lst[0]].astype(str)
    merged = pd.merge(n_personas, n_viajes, on=agg_columns_lst, how='left')
    merged['viajes_per_capita'] = merged[trip_weight] / merged[person_weight]
    return merged

#### **¿Cuál es la tasa promedio de viajes diarios en transporte público por clasificador económico?**

Las tablas y gráficos siguientes muestran los viajes per cápita en trasporte público durante días hábiles y no hábiles, por clasificador socioeconómico.

In [ ]:
print('Viajes per cápita en transporte público - Día Hábil')
df = calculate_n_viajes_per_capita(viajes_publico_habiles, "viajes_publico_habiles", "TramoIngreso", ["TramoIngreso"], 'Persona', 'PONDERADOR_CALIBRADO', 'PONDERADOR_CALIBRADO_VIAJES')
df

In [ ]:
#fig, ax = plt.subplots(figsize=(8,6))
g = sns.catplot(x="TramoIngreso", y="viajes_per_capita", 
                capsize=.2, height=4, aspect=2,
                kind="point", data=df)

g.fig.suptitle('Viajes per cápita en transporte público - Día Hábil')
g.set_ylabels('# Viajes per Cápita')
g.set_xlabels('Tramo de ingreso')
g.set_xticklabels(rotation=90)
g.set(ylim=(0,3))

#fig.tight_layout()

In [ ]:
print('Viajes per cápita en transporte público - Día No Hábil')
df = calculate_n_viajes_per_capita(viajes_publico_nohabiles, "viajes_publico_nohabiles", "TramoIngreso", ["TramoIngreso"], 'Persona', 'PONDERADOR_CALIBRADO', 'PONDERADOR_CALIBRADO_VIAJES')
df

In [ ]:
g = sns.catplot(x="TramoIngreso", y="viajes_per_capita", 
                capsize=.2, height=4, aspect=2,
                kind="point", data=df)

g.fig.suptitle('Viajes per cápita en transporte público - Día No Hábil')
g.set_ylabels('# Viajes per Cápita')
g.set_xlabels('Tramo de ingreso')
g.set_xticklabels(rotation=90)
g.set(ylim=(0,3))

#fig.tight_layout()

Para días hábiles de viaje, no se observa una diferencia para los viajes pér cápita hechos diariamente en transporte público entre los distintos tramos de ingreso. Durante días no hábiles, los viajes per cápita se mantienen similares a los realizados durante días hábiles, y solo se ve una leve disminución de viajes en los niveles medios con respecto a los otros niveles.

#### **¿Cuál es la tasa promedio de viajes diarios en transporte privado por clasificador económico?**

Las tablas y gráficos muestran los viajes per cápita en trasporte privado, durante días hábiles y no hábiles, por clasificador socioeconómico.

In [ ]:
print('Viajes per cápita en transporte privado - Día Hábil')
df = calculate_n_viajes_per_capita(viajes_privado_habiles, "viajes_privado_habiles", "TramoIngreso", ["TramoIngreso"], 'Persona', 'PONDERADOR_CALIBRADO', 'PONDERADOR_CALIBRADO_VIAJES')
df

In [ ]:
g = sns.catplot(x="TramoIngreso", y="viajes_per_capita", 
                capsize=.2, height=4, aspect=2,
                kind="point", data=df)

g.fig.suptitle('Viajes per cápita en transporte privado - Día Hábil')
g.set_ylabels('# Viajes per Cápita')
g.set_xlabels('Tramo de ingreso')
g.set_xticklabels(rotation=90)
g.set(ylim=(0,5))

#fig.tight_layout()

In [ ]:
print('Viajes per cápita en transporte privado - Día No Hábil')
df = calculate_n_viajes_per_capita(viajes_privado_nohabiles, "viajes_privado_nohabiles", "TramoIngreso", ["TramoIngreso"], 'Persona', 'PONDERADOR_CALIBRADO', 'PONDERADOR_CALIBRADO_VIAJES')
df

In [ ]:
g = sns.catplot(x="TramoIngreso", y="viajes_per_capita", 
                capsize=.2, height=4, aspect=2,
                kind="point", data=df)

g.fig.suptitle('Viajes per cápita en transporte privado - Día No Hábil')
g.set_ylabels('# Viajes per Cápita')
g.set_xlabels('Tramo de ingreso')
g.set_xticklabels(rotation=90)
g.set(ylim=(0,5))

#fig.tight_layout()

Tanto en días hábiles y no hábiles, se ve un aumento de viajes per cápita realizados en transporte privado a medida que aumenta el tramo de ingreso.

#### **¿Cuál es la razón entre los viajes en transporte público y privado por clasificador socioeconómico?**

Las tablas y gráficos a continuación muestran la razón entre los viajes en transporte público y transporte privado, durante días hábiles y no hábiles, por clasificador socioeconómico.

In [ ]:
print('Razón entre los viajes en transporte público y privado - Día Hábil')
df = viajes_privado_habiles.groupby(['TramoIngreso']).agg(n_viajes = ('PONDERADOR_CALIBRADO_VIAJES', 'sum')) / viajes_publico_habiles.groupby(['TramoIngreso']).agg(n_viajes = ('PONDERADOR_CALIBRADO_VIAJES', 'sum'))
df = df.reset_index()
df

In [ ]:
g = sns.catplot(x="TramoIngreso", y="n_viajes",
                capsize=.2, height=4, aspect=2,
                kind="point", data=df)

g.fig.suptitle('Razón entre los viajes en transporte público y privado - Día Hábil')
g.set_ylabels('Proporción de viajes privados sobre públicos')
g.set_xlabels('Tramo de Ingreso')
g.set_xticklabels(rotation=90)
g.set(ylim=(0,12))

#fig.tight_layout()

In [ ]:
print('Razón entre los viajes en transporte público y privado - Día No Hábil')
df = viajes_privado_nohabiles.groupby(['TramoIngreso']).agg(n_viajes = ('PONDERADOR_CALIBRADO_VIAJES', 'sum')) / viajes_publico_nohabiles.groupby(['TramoIngreso']).agg(n_viajes = ('PONDERADOR_CALIBRADO_VIAJES', 'sum'))
df = df.reset_index()
df

In [ ]:
g = sns.catplot(x="TramoIngreso", y="n_viajes",
                capsize=.2, height=4, aspect=2,
                kind="point", data=df)

g.fig.suptitle('Razón entre los viajes en transporte público y privado - Día No Hábil')
g.set_ylabels('Proporción de viajes privados sobre públicos')
g.set_xlabels('Tramo de Ingreso')
g.set_xticklabels(rotation=90)
g.set(ylim=(0,40))

#fig.tight_layout()

Para días hábiles y no hábiles, la razón entre viajes privados y públicos aumenta a medida que aumenta el tramo de ingreso. 

#### **¿Cuál es la distribución/partición modal de los viajes por clasificador socioeconómico?**

Las siguientes tablas y gráficos representan la partición modal de los viajes realizados en días hábiles, por clasificador socioeconómico.

In [ ]:
print('Partición Modal de los Viajes - Día Hábil')
df = viajes_habiles.groupby(['TramoIngreso', 'ModoAgregado']).sum()['PONDERADOR_CALIBRADO_VIAJES'].unstack()
#.agg(count=('MEDIO_PREDOMINANTE','count')).unstack()
df_norm = df.div(df.sum(axis=1), axis=0)
df_norm

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ax = sns.heatmap(df_norm,linewidth=0.5)

ax.set_title("Partición Modal de los Viajes - Día Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

In [ ]:
print('Partición Modal de los Viajes - Día No Hábil')
df = viajes_nohabiles.groupby(['TramoIngreso', 'ModoAgregado']).sum()['PONDERADOR_CALIBRADO_VIAJES'].unstack()
#.agg(count=('MEDIO_PREDOMINANTE','count')).unstack()
df_norm =df.div(df.sum(axis=1), axis=0)
df_norm

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ax = sns.heatmap(df_norm,linewidth=0.5)

ax.set_title("Partición Modal de los Viajes - Día No Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

Durante días hábiles y no hábiles, los modos de transporte más utilizados son el auto, el cual aumenta a medida que aumenta el tramo de ingreso, y la caminata, que disminuye al aumentar el tramo de ingreso. También destaca el auto del bus TS y de bus TS - Metro durante días hábiles, en particular en los tramos más bajos.

In [ ]:
print('Partición Modal de los Viajes de acuerdo al tipo de transporte - Día Hábil')
df = viajes_habiles.groupby(['TramoIngreso', 'ModoPriPub']).sum()['PONDERADOR_CALIBRADO_VIAJES'].unstack()
df_norm = df.div(df.sum(axis=1), axis=0)
df_norm

In [ ]:
from aves.visualization.tables import barchart

fig, ax = plt.subplots(figsize=(14, 7))

barchart(
    ax, df_norm, stacked=True, normalize=False, sort_categories=True, sort_items=False
)

ax.set_title("Partición Modal de los Viajes de acuerdo al tipo de transporte - Día Hábil")
ax.set_ylim([0, 1])
ax.set_xlabel("Tramo de ingreso")
ax.set_ylabel("Fracción de los Viajes")

fig.tight_layout()

In [ ]:
print('Partición Modal de los Viajes de acuerdo al tipo de transporte - Día No Hábil')
df = viajes_nohabiles.groupby(['TramoIngreso', 'ModoPriPub']).sum()['PONDERADOR_CALIBRADO_VIAJES'].unstack()
df_norm = df.div(df.sum(axis=1), axis=0)
df_norm

In [ ]:
from aves.visualization.tables import barchart

fig, ax = plt.subplots(figsize=(14, 7))

barchart(
    ax, df_norm, stacked=True, normalize=False, sort_categories=True, sort_items=False
)

ax.set_title("Partición Modal de los Viajes de acuerdo al tipo de transporte - Día No Hábil")
ax.set_ylim([0, 1])
ax.set_xlabel("Tramo de Ingreso")
ax.set_ylabel("Fracción de los Viajes")

fig.tight_layout()

Para tipo de transporte, se ve una disminución del uso del transporte público en general cuando se comparan días hábiles y no hábiles, y se ve que la fracción de viajes privados aumenta al aumentar el tramo de ingreso.

### Parte II: Indicadores de Tiempo de Viajes

En esta segunda parte, responderemos algunas preguntas respecto a indicadores de tiempo de viajes realizados, en días hábiles y no hábiles de viaje. Para esto, buscaremos responder las siguientes preguntas:
1. ¿Cuál es el tiempo promedio de viaje por modo y tipo de transporte?
2. ¿Cuál es el tiempo promedio de viaje en hora punta de mañana?
3. ¿Cuál es el tiempo de viaje en transporte público en hora punta de mañana?
4. ¿Cuál es el tiempo promedio de viaje al trabajo en transporte público?

In [ ]:
def weighted_median(df, val, weight):
    df_sorted = df.sort_values(val)
    cumsum = df_sorted[weight].cumsum()
    cutoff = df_sorted[weight].sum() / 2.
    return df_sorted[cumsum >= cutoff][val].iloc[0]

In [ ]:
viajes_habiles= viajes_habiles[~viajes_habiles.TiempoViaje.isna()]
viajes_nohabiles= viajes_nohabiles[~viajes_nohabiles.TiempoViaje.isna()]

#### **¿Cuál es el tiempo promedio de viaje por modo y tipo de transporte?**

A continuación, se representa el promedio y mediana en minutos de viaje por modo y tipo de transporte, en días hábiles y no hábiles, por clasificador socioeconómico.



In [ ]:
print('Duración promedio (en minutos) de viaje por modo - Dia Hábil')
df = viajes_habiles.groupby(['TramoIngreso', 'ModoAgregado']).apply(lambda x: weighted_mean(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración promedio de viaje por modo - Día Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de ingreso")

fig.tight_layout()

In [ ]:
print('Duración mediana (en minutos) de viaje por modo - Dia Hábil')
df = viajes_habiles.groupby(['TramoIngreso', 'ModoAgregado']).apply(lambda x: weighted_median(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración mediana de viaje por modo - Día Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de ingreso")

fig.tight_layout()

In [ ]:
print('Duración promedio (en minutos) de viaje por modo - Dia No Hábil')
df = viajes_nohabiles.groupby(['TramoIngreso', 'ModoAgregado']).apply(lambda x: weighted_mean(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración promedio de viaje por modo - Día No Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de ingreso")

fig.tight_layout()

In [ ]:
print('Duración mediana (en minutos) de viaje por modo - Dia No Hábil')
df = viajes_nohabiles.groupby(['TramoIngreso', 'ModoAgregado']).apply(lambda x: weighted_median(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración mediana de viaje por modo - Día No Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de ingreso")

fig.tight_layout()

Durante días hábiles y no hábiles, se observa que los tiempos promedios de viaje más altos corresponden a los modos asociados a bus TS, bus no TS y algunas combinaciones con el uso del metro, mientras que los viajes con duraciones más cortas se realizan en auto, bicicleta, caminata y taxi.

In [ ]:
print('Duración promedio (en minutos) de viaje por tipo de transporte - Dia Hábil')
df = viajes_habiles.groupby(['TramoIngreso', 'ModoPriPub']).apply(lambda x: weighted_mean(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración promedio de viaje por tipo de transporte - Día Hábil")
ax.set_xlabel("Tipo de Transporte")
ax.set_ylabel("Tramo de ingreso")

fig.tight_layout()

In [ ]:
print('Duración mediana (en minutos) de viaje por tipo de transporte - Dia Hábil')
df = viajes_habiles.groupby(['TramoIngreso', 'ModoPriPub']).apply(lambda x: weighted_median(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración mediana de viaje por tipo de transporte - Día Hábil")
ax.set_xlabel("Tipo de Transporte")
ax.set_ylabel("Tramo de ingreso")

fig.tight_layout()

In [ ]:
print('Duración promedio (en minutos) de viaje por tipo de transporte - Dia No Hábil')
df = viajes_nohabiles.groupby(['TramoIngreso', 'ModoPriPub']).apply(lambda x: weighted_mean(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración promedio de viaje por tipo de transporte - Día No Hábil")
ax.set_xlabel("Tipo de Transporte")
ax.set_ylabel("Tramo de ingreso")

fig.tight_layout()

In [ ]:
print('Duración mediana (en minutos) de viaje por tipo de transporte - Dia No Hábil')
df = viajes_nohabiles.groupby(['TramoIngreso', 'ModoPriPub']).apply(lambda x: weighted_median(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración mediana de viaje por tipo de transporte - Día No Hábil")
ax.set_xlabel("Tipo de Transporte")
ax.set_ylabel("Tramo de ingreso")

fig.tight_layout()

Al analizar por tipo de transporte, para días hábiles y no hábiles, vemos que los tiempos promedios para transporte no motorizado son los más bajos, mientras que los transportes mixtos tienen las duraciones más altas, sobre todo en los tramos más bajos de ingreso.

#### **¿Cuál es el tiempo promedio de viaje en hora punta de mañana?**

A continuación, se presentan los resultados para el promedio y mediana en minutos de viaje por modo y tipo de transporte, en días hábiles y no hábiles, por clasificador socioeconómico.

In [ ]:
print('Duración promedio (en minutos) de viaje por modo en hora punta - Dia Hábil')
df = viajes_habiles[viajes_habiles.PICO_HABIL=='Si'].groupby(['TramoIngreso', 'ModoAgregado']).apply(lambda x: weighted_mean(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración promedio de viaje hora punta mañana por modo - Día Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

In [ ]:
print('Duración mediana (en minutos) de viaje por modo en hora punta - Dia Hábil')
df = viajes_habiles[viajes_habiles.PICO_HABIL=='Si'].groupby(['TramoIngreso', 'ModoAgregado']).apply(lambda x: weighted_median(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración mediana de viaje hora punta mañana por modo - Día Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

In [ ]:
print('Duración promedio (en minutos) de viaje por modo en hora punta - Dia No Hábil')
df = viajes_nohabiles[viajes_nohabiles.PICO_NOHABIL=='Si'].groupby(['TramoIngreso', 'ModoAgregado']).apply(lambda x: weighted_mean(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración promedio de viaje hora punta mañana por modo - Día No Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

In [ ]:
print('Duración mediana (en minutos) de viaje por modo en hora punta - Dia No Hábil')
df = viajes_nohabiles[viajes_nohabiles.PICO_NOHABIL=='Si'].groupby(['TramoIngreso', 'ModoAgregado']).apply(lambda x: weighted_median(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración mediana de viaje hora punta mañana por modo - Día No Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

Para días hábiles, los medios de transporte con mayores duración de viaje en hora punta de mañana son combinaciones de transporte con Metro, o combinaciones de Bus TS con Bus No TS u otros. Para días no hábiles, nuevamente las combinaciones con Metro tienen altas duraciones de viaje, en particular Bus no TS - Metro, Otros - Metro, Otros - Bus TS - Metro. Los modos con viajes de duraciones más cortas son Caminata, Taxi y Bicicleta.

In [ ]:
print('Duración promedio (en minutos) de viaje por tipo de transporte en hora punta - Dia Hábil')
df = viajes_habiles[(viajes_habiles.PICO_HABIL=='Si')].groupby(['TramoIngreso', 'ModoPriPub']).apply(lambda x: weighted_mean(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración promedio de viaje hora punta mañana por tipo de transporte - Día Hábil")
ax.set_xlabel("Tipo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

In [ ]:
print('Duración mediana (en minutos) de viaje por tipo de transporte en hora punta - Dia Hábil')
df = viajes_habiles[(viajes_habiles.PICO_HABIL=='Si')].groupby(['TramoIngreso', 'ModoPriPub']).apply(lambda x: weighted_median(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración mediana de viaje hora punta mañana por tipo de transporte - Día Hábil")
ax.set_xlabel("Tipo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

In [ ]:
print('Duración promedio (en minutos) de viaje por tipo de transporte en hora punta - Dia No Hábil')
df = viajes_nohabiles[(viajes_nohabiles.PICO_NOHABIL=='Si')].groupby(['TramoIngreso', 'ModoPriPub']).apply(lambda x: weighted_mean(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración promedio de viaje hora punta mañana por tipo de transporte - Día No Hábil")
ax.set_xlabel("Tipo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

In [ ]:
print('Duración mediana (en minutos) de viaje por tipo de transporte en hora punta - Dia No Hábil')
df = viajes_nohabiles[(viajes_nohabiles.PICO_NOHABIL=='Si')].groupby(['TramoIngreso', 'ModoPriPub']).apply(lambda x: weighted_median(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración mediana de viaje hora punta mañana por tipo de transporte - Día No Hábil")
ax.set_xlabel("Tipo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

Para días hábiles y no hábiles, los viajes con menor duración en hora punta de mañana son los realizados en transporte no motorizado, y los de mayor duración son los realizados en transporte mixto (con duración máxima en el tramo de ingreso más bajo), seguido por transporte público.

#### **¿Cuál es el tiempo de viaje en transporte público en hora punta de mañana?**

A continuación, se presentan los resultados para duración promedio de viaje en hora punta de mañana en transporte público, en días hábiles y no hábiles, por clasificador socioeconómico.

In [ ]:
print('Duración promedio (en minutos) de viaje por modo en hora punta en transporte público - Dia Hábil')
mask = (viajes_habiles.ModoPriPub=='Publico')
df = viajes_habiles[viajes_habiles.PICO_HABIL=='Si'][mask].groupby(['TramoIngreso', 'ModoAgregado']).apply(lambda x: weighted_mean(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración promedio (en minutos) de viaje en hora punta en transporte público - Dia Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

In [ ]:
print('Duración mediana (en minutos) de viaje por modo en hora punta en transporte público - Dia Hábil')
mask = (viajes_habiles.ModoPriPub=='Publico')
df = viajes_habiles[viajes_habiles.PICO_HABIL=='Si'][mask].groupby(['TramoIngreso', 'ModoAgregado']).apply(lambda x: weighted_median(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración mediana (en minutos) de viaje en hora punta en transporte público - Dia Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

In [ ]:
print('Duración promedio (en minutos) de viaje hora punta en transporte público - Dia No Hábil')
mask = (viajes_nohabiles.ModoPriPub=='Publico')
df = viajes_nohabiles[viajes_nohabiles.PICO_NOHABIL=='Si'][mask].groupby(['TramoIngreso', 'ModoAgregado']).apply(lambda x: weighted_mean(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración promedio (en minutos) de viaje en hora punta en transporte público - Dia No Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

In [ ]:
print('Duración mediana (en minutos) de viaje hora punta en transporte público - Dia No Hábil')
mask = (viajes_nohabiles.ModoPriPub=='Publico')
df = viajes_nohabiles[viajes_nohabiles.PICO_NOHABIL=='Si'][mask].groupby(['TramoIngreso', 'ModoAgregado']).apply(lambda x: weighted_median(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración mediana (en minutos) de viaje en hora punta en transporte público - Dia No Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

En días hábiles y no hábiles, las duraciones mayores de viaje en transporte público en hora punta de mañana son de las combinaciones de Bus TS - Bus no TS, Bus no TS - Metro, Otros - Bus TS - Metro y Otros - Metro, con estas dos últimas opciones con valores más altos en los tramos más bajos. Las duraciones más bajas corresponden a Taxi Colectivo a través de todos los tramos.

#### **¿Cuál es el tiempo promedio de viaje al trabajo en transporte público?**

A continuación, se presentan los resultados de tiempo promedio de viaje al trabajo en transporte público, por clasificador socioeconómico.

In [ ]:
viajes_habiles.PropositoAgregado.unique()

In [ ]:
print('Duración promedio (en minutos) de viaje al trabajo en transporte público - Dia Hábil')
mask = (viajes_habiles.ModoPriPub=='Publico') & (viajes_habiles.PropositoAgregado=='Trabajo')
df = viajes_habiles[mask].groupby(['TramoIngreso', 'ModoAgregado']).apply(lambda x: weighted_mean(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración promedio (en minutos) de viaje al trabajo en transporte público - Dia Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

In [ ]:
print('Duración mediana (en minutos) de viaje al trabajo en transporte público - Dia Hábil')
mask = (viajes_habiles.ModoPriPub=='Publico') & (viajes_habiles.PropositoAgregado=='Trabajo')
df = viajes_habiles[mask].groupby(['TramoIngreso', 'ModoAgregado']).apply(lambda x: weighted_median(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración mediana (en minutos) de viaje al trabajo en transporte público - Dia Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

In [ ]:
print('Duración promedio (en minutos) de viaje al trabajo en transporte público - Dia No Hábil')
mask = (viajes_nohabiles.ModoPriPub=='Publico') & (viajes_nohabiles.PropositoAgregado=='Trabajo')
df = viajes_nohabiles[mask].groupby(['TramoIngreso', 'ModoAgregado']).apply(lambda x: weighted_mean(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración promedio (en minutos) de viaje al trabajo en transporte público - Dia No Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

In [ ]:
print('Duración mediana (en minutos) de viaje al trabajo en transporte público - Dia No Hábil')
mask = (viajes_nohabiles.ModoPriPub=='Publico') & (viajes_nohabiles.PropositoAgregado=='Trabajo')
df = viajes_nohabiles[mask].groupby(['TramoIngreso', 'ModoAgregado']).apply(lambda x: weighted_median(x, 'TiempoViaje', 'PONDERADOR_CALIBRADO_VIAJES')).unstack()
df

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

ax = sns.heatmap(df, annot=True)

ax.set_title("Duración mediana (en minutos) de viaje al trabajo en transporte público - Dia No Hábil")
ax.set_xlabel("Modo de Transporte")
ax.set_ylabel("Tramo de Ingreso")

fig.tight_layout()

En días hábiles y no hábiles, los modos con mayor duración de viaje al trabajo en transporte público son de las combinaciones de Bus TS - Bus no TS, Bus no TS - Metro y Otros - Bus TS - Metro, con fluctuaciones de variación a través de los tramos de ingreso. Las duraciones más bajas corresponden a Taxi Colectivo a través de todos los tramos.

### Parte III: Indicadores de Distancia de Viajes
1. Distancia de viajes en auto (histograma de viajes por km)
2.Distancia de viajes en transporte público (histograma de viajes por km)
3.Distancia de viajes por motivo estudio (histograma de viajes por km)
4.Distancia de viajes por motivo al trabajo (histograma de viajes por km)
5.Viajes interzonales como intrazonales


En esta sección, se presentarán los indicadores de distancia de viajes durante días hábiles.

In [ ]:
viajes_habiles = viajes_habiles[(viajes_habiles['DistManhattan'].between(500, 45000))]
viajes_nohabiles = viajes_nohabiles[(viajes_nohabiles['DistManhattan'].between(500, 45000))]
viajes_personas = viajes_personas[(viajes_personas['DistManhattan'].between(500, 45000))]


#### **Distancia de viajes en auto (histograma de viajes por km)**

In [ ]:
from matplotlib.pyplot import hist
print('Distancia de viajes en auto - Día Hábil')
mask = (viajes_habiles.ModoAgregado == 'Auto')
df = viajes_habiles[mask].groupby('DistManhattan').sum()[['PONDERADOR_CALIBRADO_VIAJES']].reset_index().sort_values('DistManhattan')
hist(df.DistManhattan, weights=df.PONDERADOR_CALIBRADO_VIAJES, bins=50)
plt.axvline(viajes_habiles[mask].groupby('Viaje').apply(lambda x: weighted_mean(x, 'DistManhattan', 'PONDERADOR_CALIBRADO_VIAJES')).mean(), color='green', linestyle='dashed', linewidth=1)
plt.axvline(viajes_habiles[mask].groupby('Viaje').apply(lambda x: weighted_median(x, 'DistManhattan', 'PONDERADOR_CALIBRADO_VIAJES')).median(), color='red', linestyle='dashed', linewidth=1)
#Verde Promedio
#Rojo Mediana

In [ ]:
print('Distancia de viajes en auto - Día No Hábil')
mask = (viajes_nohabiles.ModoAgregado == 'Auto')
df = viajes_nohabiles[mask].groupby('DistManhattan').sum()[['PONDERADOR_CALIBRADO_VIAJES']].reset_index().sort_values('DistManhattan')
hist(df.DistManhattan, weights=df.PONDERADOR_CALIBRADO_VIAJES, bins=50)
plt.axvline(viajes_nohabiles[mask].groupby('Viaje').apply(lambda x: weighted_mean(x, 'DistManhattan', 'PONDERADOR_CALIBRADO_VIAJES')).mean(), color='green', linestyle='dashed', linewidth=1)
plt.axvline(viajes_nohabiles[mask].groupby('Viaje').apply(lambda x: weighted_median(x, 'DistManhattan', 'PONDERADOR_CALIBRADO_VIAJES')).median(), color='red', linestyle='dashed', linewidth=1)
#Verde Promedio
#Rojo Mediana

#### **Distancia de viajes en transporte público**

In [ ]:
print('Distancia de viajes en transporte público - Día Hábil')
mask = (viajes_habiles.ModoPriPub == 'Publico')
df = viajes_habiles[mask].groupby('DistManhattan').sum()[['PONDERADOR_CALIBRADO_VIAJES']].reset_index().sort_values('DistManhattan')
hist(df.DistManhattan, weights=df.PONDERADOR_CALIBRADO_VIAJES, bins=50)
plt.axvline(viajes_habiles[mask].groupby('Viaje').apply(lambda x: weighted_mean(x, 'DistManhattan', 'PONDERADOR_CALIBRADO_VIAJES')).mean(), color='green', linestyle='dashed', linewidth=1)
plt.axvline(viajes_habiles[mask].groupby('Viaje').apply(lambda x: weighted_median(x, 'DistManhattan', 'PONDERADOR_CALIBRADO_VIAJES')).median(), color='red', linestyle='dashed', linewidth=1)
#Verde Promedio
#Rojo Mediana

In [ ]:
print('Distancia de viajes en transporte público - Día No Hábil')
mask = (viajes_nohabiles.ModoPriPub == 'Publico')
df = viajes_nohabiles[mask].groupby('DistManhattan').sum()[['PONDERADOR_CALIBRADO_VIAJES']].reset_index().sort_values('DistManhattan')
hist(df.DistManhattan, weights=df.PONDERADOR_CALIBRADO_VIAJES, bins=50)
plt.axvline(viajes_nohabiles[mask].groupby('Viaje').apply(lambda x: weighted_mean(x, 'DistManhattan', 'PONDERADOR_CALIBRADO_VIAJES')).mean(), color='green', linestyle='dashed', linewidth=1)
plt.axvline(viajes_nohabiles[mask].groupby('Viaje').apply(lambda x: weighted_median(x, 'DistManhattan', 'PONDERADOR_CALIBRADO_VIAJES')).median(), color='red', linestyle='dashed', linewidth=1)
#Verde Promedio
#Rojo Mediana

#### **Distancia de viajes por motivo estudio**

In [ ]:
print('Distancia de viajes con motivo de estudio')
mask = (viajes_personas.PropositoAgregado=='Estudio')
df = viajes_personas[mask].groupby('DistManhattan').sum()[['PONDERADOR_CALIBRADO_VIAJES']].reset_index().sort_values('DistManhattan')
hist(df.DistManhattan, weights=df.PONDERADOR_CALIBRADO_VIAJES, bins=50)
plt.axvline(viajes_personas[mask].groupby('Viaje').apply(lambda x: weighted_mean(x, 'DistManhattan', 'PONDERADOR_CALIBRADO_VIAJES')).mean(), color='green', linestyle='dashed', linewidth=1)
plt.axvline(viajes_personas[mask].groupby('Viaje').apply(lambda x: weighted_median(x, 'DistManhattan', 'PONDERADOR_CALIBRADO_VIAJES')).median(), color='red', linestyle='dashed', linewidth=1)
#Verde Promedio
#Rojo Mediana

#### **Distancia de viajes por motivo trabajo**

In [ ]:
print('Distancia de viajes con motivo de trabajo')
mask = (viajes_personas.PropositoAgregado=='Trabajo')
df = viajes_personas[mask].groupby('DistManhattan').sum()[['PONDERADOR_CALIBRADO_VIAJES']].reset_index().sort_values('DistManhattan')
hist(df.DistManhattan, weights=df.PONDERADOR_CALIBRADO_VIAJES, bins=50)
plt.axvline(viajes_personas[mask].groupby('Viaje').apply(lambda x: weighted_mean(x, 'DistManhattan', 'PONDERADOR_CALIBRADO_VIAJES')).mean(), color='green', linestyle='dashed', linewidth=1)
plt.axvline(viajes_personas[mask].groupby('Viaje').apply(lambda x: weighted_median(x, 'DistManhattan', 'PONDERADOR_CALIBRADO_VIAJES')).median(), color='red', linestyle='dashed', linewidth=1)
#Verde Promedio
#Rojo Mediana

#### **Viajes intra vs interzonales**

In [ ]:
print('% de viajes Inter e Intra zonales')
df = viajes_personas.groupby(['PropositoAgregado','Sexo','Intra_Inter_zona']).sum()['PONDERADOR_CALIBRADO_VIAJES'].unstack()
df.div(df.sum(axis=1), axis=0)

In [ ]:
print('% de viajes Inter e Intra comunales')
df = viajes_personas.groupby(['PropositoAgregado','Sexo','Intra_Inter_comuna']).sum()['PONDERADOR_CALIBRADO_VIAJES'].unstack()
df.div(df.sum(axis=1), axis=0)

In [ ]:
print('% de viajes Inter e Intra sector')
df = viajes_personas.groupby(['PropositoAgregado','Sexo','Intra_Inter_sector']).sum()['PONDERADOR_CALIBRADO_VIAJES'].unstack()
df.div(df.sum(axis=1), axis=0)

### ¿Dónde se concentran las personas que utilizan cada modo de transporte en la ciudad para distintos propósitos?

In [ ]:
from aves.features.geo import clip_area_geodataframe
bbox = [326815.3485,6275131.9514,365914.0887,6312532.9784]

zonas_en_caja = clip_area_geodataframe(city_shp.to_crs('epsg:32719'), bbox)
zonas_en_caja.plot()

In [ ]:
bounds = zonas_en_caja.to_crs('EPSG:4686').total_bounds

In [ ]:
import contextily as cx

scl_img, scl_ext = cx.bounds2raster(bounds[0], bounds[1], bounds[2], bounds[3], 
    "santiago_toner_12.tif",
    ll=True,
    source=cx.providers.Stamen.TonerBackground,
    zoom=12,
)

In [ ]:
viajes_personas.columns

In [ ]:
viajes_personas.Proposito.unique()

In [ ]:
def transform_motives(row):
  if row['Proposito'] in ['Al trabajo', 'Por trabajo']:
    return 'Trabajo'
  elif row['Proposito']in ['Al estudio', 'Por estudio']:
    return 'Estudios'
  else:
    return row['Proposito']

In [ ]:
viajes_personas['proposito'] = viajes_personas.apply(lambda x: transform_motives(x), axis=1)

In [ ]:
viajes_personas.ModoAgregado.unique()

In [ ]:
from aves.features.geo import to_point_geodataframe
origenes_viajes = to_point_geodataframe(viajes_personas, 'OrigenCoordX' , 'OrigenCoordY', crs='epsg:32719')
destinos_viajes = to_point_geodataframe(viajes_personas, 'DestinoCoordX', 'DestinoCoordY', crs='epsg:32719')

In [ ]:
from aves.features.geo import clip_point_geodataframe

origenes_viajes = origenes_viajes[(origenes_viajes['Viaje'].isin(destinos_viajes['Viaje']))]
origenes_viajes = clip_point_geodataframe(origenes_viajes, zonas_en_caja.total_bounds)
destinos_viajes = destinos_viajes[(destinos_viajes['Viaje'].isin(origenes_viajes['Viaje']))]
destinos_viajes = clip_point_geodataframe(destinos_viajes, zonas_en_caja.total_bounds)

In [ ]:
from aves.visualization.figures import GeoFacetGrid

from aves.visualization.maps import heat_map

grid = GeoFacetGrid(
    origenes_viajes,
    context=zonas_en_caja,
    row="proposito",
    col="ModoAgregado",
    row_order=["Trabajo", "Estudios"],
    col_order=['Bus TS', 'Bus TS - Metro', 'Auto', 'Caminata'],
    height=6,
    hue="ModoAgregado"
)
grid.add_basemap("santiago_toner_12.tif")
#grid.add_layer(city_shp_filt, color="#efefef", edgecolor="white", linewidth=1)

grid.add_layer(
    heat_map,
    # atributo de los datos con la importancia o peso de cada viaje
    weight="PONDERADOR_CALIBRADO_VIAJES",
    # cantidad de niveles/colores del mapa de calor
    n_levels=10,
    # radio de influencia de cada viaje
    bandwidth=1000,
    # valor de corte para los valores bajos del heatmap
    low_threshold=0.075,
    # transparencia
    alpha=0.75,
    # paleta de colores
    palette="inferno"
)

grid.add_global_colorbar('inferno', 10, title='Intensidad de Viajes (de menos a más)', orientation='horizontal')
#grid.set_title("Viajes a trabajar y a estudiar de acuerdo al modo de transporte")
grid.fig.tight_layout()

In [ ]:
grid = GeoFacetGrid(
    destinos_viajes[destinos_viajes.proposito=='volver a casa'],
    context=zonas_en_caja,
    #row="MOTIVOVIAJE",
    col="ModoAgregado",
    col_wrap=3,
    row_order=["volver a casa"],
    col_order=['Bus TS', 'Bus TS - Metro', 'Auto', 'Caminata'],
    height=5,
    hue="ModoAgregado"
)
grid.add_basemap("santiago_toner_12.tif")
#grid.add_layer(city_shp_filt, color="#efefef", edgecolor="white", linewidth=1, alpha=0.5)

grid.add_layer(
    heat_map,
    # atributo de los datos con la importancia o peso de cada viaje
    weight="PONDERADOR_CALIBRADO_VIAJES",
    # cantidad de niveles/colores del mapa de calor
    n_levels=10,
    # radio de influencia de cada viaje
    bandwidth=0.005,
    # valor de corte para los valores bajos del heatmap
    low_threshold=0.075,
    # transparencia
    alpha=0.75,
    # paleta de colores
    palette="inferno"
)
grid.add_global_colorbar('inferno', 10, title='Intensidad de Viajes (de menos a más)', orientation='horizontal')
grid.set_title("Viajes de vuelta a casa en Santiago")
grid.fig.tight_layout()

#### ¿Cuán lejos queda el trabajo de acuerdo al lugar de residencia?

Con esta pregunta queremos entender si existe un patrón geográfico en las elecciones de residencia y trabajo de las personas.

Para responder la pregunta, primero filtramos los viajes que nos interesan:

In [ ]:
viajes_todos = origenes_viajes[(pd.notnull(origenes_viajes.PONDERADOR_CALIBRADO_VIAJES)) &
                                (origenes_viajes.DistManhattan > 0)].drop_duplicates(subset=['Persona'], keep='first')

In [ ]:
viajes_habiles = origenes_viajes[(origenes_viajes.DIA_HABIL == 'Si') &
                                (pd.notnull(origenes_viajes.PONDERADOR_CALIBRADO_VIAJES)) &
                                (origenes_viajes.DistManhattan > 0)].drop_duplicates(subset=['Persona'], keep='first')

In [ ]:
viajes_trabajo = origenes_viajes[(origenes_viajes.proposito == 'Trabajo') &
                                (pd.notnull(origenes_viajes.PONDERADOR_CALIBRADO_VIAJES)) &
                                (origenes_viajes.DistManhattan > 0)].drop_duplicates(subset=['Persona'], keep='first')
                                
print(len(viajes_trabajo), viajes_trabajo.PONDERADOR_CALIBRADO_VIAJES.sum())

In [ ]:
distancia_zonas_mean = (viajes_trabajo
                   .groupby(['ZonaOrigen'])
                   .apply(lambda x: weighted_mean(x, 'DistManhattan', 'PONDERADOR_CALIBRADO_VIAJES'))
                   .rename('media_distancia_al_trabajo')
)

In [ ]:
from aves.visualization.maps import choropleth_map
grid = GeoFacetGrid(zonas_en_caja.set_index('Zona').join(distancia_zonas_mean, how="left"), height=9)
grid.add_basemap("santiago_toner_12.tif")
grid.add_layer(
    choropleth_map,
    "media_distancia_al_trabajo",
    k=5,
    linewidth=0.5,
    edgecolor="black",
    binning="fisher_jenks",
    palette="RdPu",
    alpha=0.85,
    cbar_args=dict(
        label="Distancia (m)",
        height="22%",
        width="2%",
        orientation="vertical",
        location="center left",
        label_size="small",
        bbox_to_anchor=(0.0, 0.0, 0.9, 1.0),
    ),
)
grid.add_map_elements()
grid.set_title("Distancia al Trabajo Promedio de acuerdo a la Zona de Origen")
grid.tight_layout()

In [ ]:
matriz_zonas = (viajes_trabajo[(viajes_trabajo['ZonaOrigen'] != viajes_trabajo['ZonaDestino'])
                            
                             & (viajes_trabajo['ZonaOrigen'].isin(zonas_en_caja.Zona))
                             & (viajes_trabajo['ZonaDestino'].isin(zonas_en_caja.Zona))]
                    .groupby(['ZonaOrigen', 'ZonaDestino'])
                    .agg(n_viajes=('PONDERADOR_CALIBRADO_VIAJES', 'sum'))
                    .sort_values('n_viajes', ascending=False)
                    .assign(cumsum_viajes=lambda x: x['n_viajes'].cumsum())
                    .assign(cumsum_viajes=lambda x: x['cumsum_viajes'] / x['cumsum_viajes'].max())
                    .reset_index()
)
matriz_zonas.to_csv(data_path + 'matriz_zonas_trabajo_santiago2012.csv', index=False)

In [ ]:
matriz_zonas = (viajes_todos[(viajes_todos['ZonaOrigen'] != viajes_todos['ZonaDestino'])
                             & (viajes_todos['ZonaOrigen'].isin(zonas_en_caja.Zona))
                             & (viajes_todos['ZonaDestino'].isin(zonas_en_caja.Zona))]
                    .groupby(['ZonaOrigen', 'ZonaDestino'])
                    .agg(n_viajes=('PONDERADOR_CALIBRADO_VIAJES', 'sum'))
                    .sort_values('n_viajes', ascending=False)
                    .assign(cumsum_viajes=lambda x: x['n_viajes'].cumsum())
                    .assign(cumsum_viajes=lambda x: x['cumsum_viajes'] / x['cumsum_viajes'].max())
                    .reset_index()
)
matriz_zonas.to_csv(data_path + 'matriz_zonas_todos_santiago2012.csv', index=False)

In [ ]:
matriz_zonas = (viajes_habiles[(viajes_habiles['ZonaOrigen'] != viajes_habiles['ZonaDestino'])
                             & (viajes_habiles['ZonaOrigen'].isin(zonas_en_caja.Zona))
                             & (viajes_habiles['ZonaDestino'].isin(zonas_en_caja.Zona))]
                    .groupby(['ZonaOrigen', 'ZonaDestino'])
                    .agg(n_viajes=('PONDERADOR_CALIBRADO_VIAJES', 'sum'))
                    .sort_values('n_viajes', ascending=False)
                    .assign(cumsum_viajes=lambda x: x['n_viajes'].cumsum())
                    .assign(cumsum_viajes=lambda x: x['cumsum_viajes'] / x['cumsum_viajes'].max())
                    .reset_index()
)
matriz_zonas.to_csv(data_path + 'matriz_zonas_habiles_santiago2012.csv', index=False)

In [ ]:
matriz_zonas = matriz_zonas[matriz_zonas['cumsum_viajes'] <= 0.8]

In [ ]:
merged_zones = zonas_en_caja.dissolve('Zona')

In [ ]:
from aves.models.network import Network
from aves.visualization.networks import NodeLink

zone_od_network = Network.from_edgelist(
    matriz_zonas, source="ZonaOrigen", target="ZonaDestino", weight="n_viajes"
)

In [ ]:
zone_nodelink = NodeLink(zone_od_network)
zone_nodelink.layout_nodes(method="geographical", geodataframe=merged_zones)
zone_nodelink.set_node_drawing("plain", weights=zone_od_network.node_degree("in"))
zone_nodelink.set_edge_drawing(method="origin-destination")

In [ ]:
def plot_network(ax, geo_data, *args, **kwargs):
    zone_nodelink.plot(ax, *args, **kwargs)

In [ ]:
zone_nodelink.bundle_edges(
    method="force-directed", K=1, S=500, I=30, compatibility_threshold=0.65, C=6
)

In [ ]:
grid = GeoFacetGrid(zonas_en_caja, height=13)
grid.add_basemap("santiago_toner_12.tif")
#grid.add_layer(city_shp_filt,facecolor='white', edgecolor='grey', alpha=0.25)
grid.add_layer(
    plot_network,
    nodes=dict(color="white", edgecolor="black", node_size=100, alpha=0.95),
    edges=dict(linewidth=0.5, alpha=0.55),
)
grid.set_title("Viajes al trabajo en Santiago")

## Coeficiente de Movilidad

In [ ]:
viajes_personas['duracion_minutos'] = viajes_personas['TiempoViaje']

In [ ]:
q1 = '''SELECT Persona, Sexo
, PONDERADOR_CALIBRADO 
,  count(*) as n_viajes, AVG(duracion_minutos) as tiempo_total 
       FROM viajes_personas where duracion_minutos < 150 group by 1,2,3'''

n_viajes = sqldf(q1, locals())

In [ ]:
eod.plot_lmplot(n_viajes)

In [ ]:
groups = eod.generate_groups(n_viajes)
print(eod.calculate_indicators(groups,'PONDERADOR_CALIBRADO'))

In [ ]:
eod.plot_lmplot(n_viajes, col="Sexo", hue="Sexo", col_wrap=2)

In [ ]:
subgroups = n_viajes.Sexo.unique()
for element in subgroups:
    print(element)
    groups = eod.generate_groups(n_viajes[n_viajes.Sexo==element])
    print(eod.calculate_indicators(groups,'PONDERADOR_CALIBRADO'))